# Medicaid spending on prescription drugs

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.float_format", lambda x: "%.2f" % x) # Suppress scientific notation
# Enable browser notifications
%load_ext jupyternotify

<IPython.core.display.Javascript object>

## Import [data](https://www.nasbo.org/mainsite/reports-data/state-expenditure-report) on state expenditures from NASBO

In [2]:
budgets = pd.read_excel("data/medicaid_spending/state_expenditures.xlsx", usecols=[0, 1, 22, 23, 24, 25, 26, 72, 73, 74, 75, 76], names=["year", "state", "medicaid_general", "medicaid_federal", "medicaid_other", "medicaid_bonds", "medicaid_total", "all_spending_general", "all_spending_federal", "all_spending_other", "all_spending_bonds", "all_spending_total"], sheet_name="State Exp Report Data")
budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1376 entries, 0 to 1375
Data columns (total 12 columns):
year                    1376 non-null int64
state                   1376 non-null object
medicaid_general        1355 non-null float64
medicaid_federal        1354 non-null float64
medicaid_other          1355 non-null float64
medicaid_bonds          1354 non-null float64
medicaid_total          1376 non-null float64
all_spending_general    1376 non-null float64
all_spending_federal    1376 non-null float64
all_spending_other      1376 non-null float64
all_spending_bonds      1376 non-null float64
all_spending_total      1376 non-null float64
dtypes: float64(10), int64(1), object(1)
memory usage: 129.1+ KB


Filter the data to years for 2008 and later.

In [3]:
budgets = budgets[budgets["year"] >= 2008].reset_index(drop=True)
budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 12 columns):
year                    510 non-null int64
state                   510 non-null object
medicaid_general        500 non-null float64
medicaid_federal        500 non-null float64
medicaid_other          500 non-null float64
medicaid_bonds          500 non-null float64
medicaid_total          510 non-null float64
all_spending_general    510 non-null float64
all_spending_federal    510 non-null float64
all_spending_other      510 non-null float64
all_spending_bonds      510 non-null float64
all_spending_total      510 non-null float64
dtypes: float64(10), int64(1), object(1)
memory usage: 47.9+ KB


In [4]:
budgets.head(1)

,year,state,medicaid_general,medicaid_federal,medicaid_other,medicaid_bonds,medicaid_total,all_spending_general,all_spending_federal,all_spending_other,all_spending_bonds,all_spending_total
0,2008,Alabama,471.00,2899.00,1030.00,0.00,4400.00,8460.00,6291.00,4537.00,552.00,19840.00


Create a column that combines all state funding sources (general, other and bonds).

In [5]:
budgets["medicaid_state"] = budgets["medicaid_general"] + budgets["medicaid_other"] + budgets["medicaid_bonds"]
budgets["all_spending_state"] = budgets["all_spending_general"] + budgets["all_spending_other"] + budgets["all_spending_bonds"]
budgets.drop(["medicaid_general", "medicaid_other", "medicaid_bonds"], axis=1, inplace=True) # Drop component columns
budgets.drop(["all_spending_general", "all_spending_other", "all_spending_bonds"], axis=1, inplace=True) # Drop component columns
budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 8 columns):
year                    510 non-null int64
state                   510 non-null object
medicaid_federal        500 non-null float64
medicaid_total          510 non-null float64
all_spending_federal    510 non-null float64
all_spending_total      510 non-null float64
medicaid_state          500 non-null float64
all_spending_state      510 non-null float64
dtypes: float64(6), int64(1), object(1)
memory usage: 32.0+ KB


Spot check the post-calculation numbers.

In [6]:
budgets.head(1)

,year,state,medicaid_federal,medicaid_total,all_spending_federal,all_spending_total,medicaid_state,all_spending_state
0,2008,Alabama,2899.00,4400.00,6291.00,19840.00,1501.00,13549.00


## Import the CMS [Medicaid drug utilization data](https://www.medicaid.gov/medicaid/prescription-drugs/state-drug-utilization-data/index.html)

Query the Medicaid drug utilization API for each year and calculate the Medicaid amount paid.

In [7]:
query_16 = "https://data.medicaid.gov/resource/4kp3-zsqr.json?$select=period_covered,sum(medicaid_amount_reimbursed)&$group=state_code,period_covered"
drug_spending_16 = pd.read_json(query_16)
query_15 = "https://data.medicaid.gov/resource/h5ii-2ar3.json?$select=period_covered,sum(medicaid_amount_reimbursed)&$group=state_code,period_covered"
drug_spending_15 = pd.read_json(query_15)
query_14 = "https://data.medicaid.gov/resource/hz5c-g52b.json?$select=period_covered,sum(medicaid_amount_reimbursed)&$group=state_code,period_covered"
drug_spending_14 = pd.read_json(query_14)
query_13 = "https://data.medicaid.gov/resource/kmyc-3n7k.json?$select=period_covered,sum(medicaid_amount_reimbursed)&$group=state_code,period_covered"
drug_spending_13 = pd.read_json(query_13)
query_12 = "https://data.medicaid.gov/resource/mkpf-yey3.json?$select=period_covered,sum(medicaid_amount_reimbursed)&$group=state_code,period_covered"
drug_spending_12 = pd.read_json(query_12)
query_11 = "https://data.medicaid.gov/resource/de79-47w8.json?$select=period_covered,sum(medicaid_amount_reimbursed)&$group=state_code,period_covered"
drug_spending_11 = pd.read_json(query_11)
query_10 = "https://data.medicaid.gov/resource/dgte-3gu8.json?$select=period_covered,sum(medicaid_amount_reimbursed)&$group=state_code,period_covered"
drug_spending_10 = pd.read_json(query_10)
query_09 = "https://data.medicaid.gov/resource/6tcm-8x84.json?$select=period_covered,sum(medicaid_amount_reimbursed)&$group=state_code,period_covered"
drug_spending_09 = pd.read_json(query_09)
query_08 = "https://data.medicaid.gov/resource/bzcm-rrj6.json?$select=period_covered,sum(medicaid_amount_reimbursed)&$group=state_code,period_covered"
drug_spending_08 = pd.read_json(query_08)

Concatenate the dataframes into a single dataframe of spending in all years.

In [8]:
drug_spending = pd.concat([drug_spending_16, drug_spending_15, drug_spending_14, drug_spending_13, drug_spending_12, drug_spending_11, drug_spending_10, drug_spending_09, drug_spending_08]).reset_index(drop=True)
drug_spending.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
period_covered                    9 non-null int64
sum_medicaid_amount_reimbursed    9 non-null float64
dtypes: float64(1), int64(1)
memory usage: 224.0 bytes


In [9]:
drug_spending.rename(columns={"period_covered": "year", "sum_medicaid_amount_reimbursed": "drug_spending"}, inplace=True)
drug_spending

,year,drug_spending
0,2016,62016683226.54
1,2015,55620114965.09
2,2014,47078114991.53
3,2013,37940450109.63
4,2012,37976860399.61
5,2011,37783429301.78
6,2010,32989906941.56
7,2009,26014836021.22
8,2008,24642404374.76


## Import the [Medicaid drug rebates data](https://www.medicaid.gov/medicaid/finance/state-expenditure-reporting/expenditure-reports/index.html) from the CMS MBES/CBES reports

In [10]:
drug_rebates_16 = pd.read_csv("data/medicaid_spending/FY_2016_Financial_Management_Data_-_National_Totals.csv", header=0, usecols=[0, 3, 4], names=["year", "service_category", "drug_rebates"])
drug_rebates_15 = pd.read_excel("data/medicaid_spending/FY 2015 NET EXPENDITURES.xlsx", sheet_name="MAP - National Totals", header=6, usecols=[0, 1], names=["service_category", "drug_rebates"])
drug_rebates_14 = pd.read_excel("data/medicaid_spending/FMR Net Expenditures FY14.xlsx", sheet_name="MAP - National Totals", header=6, usecols=[0, 1], names=["service_category", "drug_rebates"])
drug_rebates_13 = pd.read_excel("data/medicaid_spending/FMR Net Expenditures FY13.xlsx", sheet_name="MAP - National Totals", header=6, usecols=[0, 1], names=["service_category", "drug_rebates"])
drug_rebates_12 = pd.read_excel("data/medicaid_spending/FMR Net Expenditures FY12.xlsx", sheet_name="National Totals", header=6, usecols=[0, 1], names=["service_category", "drug_rebates"])
# For the following imports, we will skip all the rows until the "National Totals" data begins
drug_rebates_11 = pd.read_excel("data/medicaid_spending/NetExpenditure02through11.xlsx", sheet_name="2011", skiprows=10605, header=7, usecols=[0, 1], names=["service_category", "drug_rebates"]) 
drug_rebates_10 = pd.read_excel("data/medicaid_spending/NetExpenditure02through11.xlsx", sheet_name="2010", skiprows=10031, header=7, usecols=[0, 1], names=["service_category", "drug_rebates"]) 
drug_rebates_09 = pd.read_excel("data/medicaid_spending/NetExpenditure02through11.xlsx", sheet_name="2009", skiprows=7303, header=7, usecols=[0, 1], names=["service_category", "drug_rebates"]) 
drug_rebates_08 = pd.read_excel("data/medicaid_spending/NetExpenditure02through11.xlsx", sheet_name="2008", skiprows=7175, header=7, usecols=[0, 1], names=["service_category", "drug_rebates"]) 

Add a year column to the 2008 through 2015 dataframes.

In [11]:
drug_rebates_15["year"] = 2015
drug_rebates_14["year"] = 2014
drug_rebates_13["year"] = 2013
drug_rebates_12["year"] = 2012
drug_rebates_11["year"] = 2011
drug_rebates_10["year"] = 2010
drug_rebates_09["year"] = 2009
drug_rebates_08["year"] = 2008

Convert the drug rebates column to numeric (int) data type in the 2016 data.

In [12]:
drug_rebates_16["drug_rebates"] = drug_rebates_16["drug_rebates"].str.replace(",", "")
drug_rebates_16["drug_rebates"] = pd.to_numeric(drug_rebates_16["drug_rebates"])
drug_rebates_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 3 columns):
year                277 non-null int64
service_category    277 non-null object
drug_rebates        277 non-null int64
dtypes: int64(2), object(1)
memory usage: 6.6+ KB


Concatenate the dataframes into a single dataframe of rebates in all years.

In [13]:
drug_rebates = pd.concat([drug_rebates_16, drug_rebates_15, drug_rebates_14, drug_rebates_13, drug_rebates_12, drug_rebates_11, drug_rebates_10, drug_rebates_09, drug_rebates_08]).reset_index(drop=True)
drug_rebates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1739 entries, 0 to 1738
Data columns (total 3 columns):
drug_rebates        1710 non-null float64
service_category    1724 non-null object
year                1739 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 40.8+ KB


/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


Filter the data to just the six drug rebates.

In [14]:
drug_rebates = drug_rebates[(drug_rebates["service_category"].str.startswith("Drug Rebate Offset - ")) | (drug_rebates["service_category"].str.startswith("MCO - ")) | (drug_rebates["service_category"].str.startswith("Increased ACA OFFSET - "))]
drug_rebates

,drug_rebates,service_category,year
75,-12885689016.00,Drug Rebate Offset - National,2016
76,-864806347.00,Drug Rebate Offset - State Sidebar Agreement,2016
77,-15859240388.00,MCO - National Agreement,2016
78,-230435858.00,MCO - State Sidebar Agreement,2016
79,-576527028.00,Increased ACA OFFSET - Fee for Service,2016
80,-776169673.00,Increased ACA OFFSET - MCO,2016
295,-10547657885.00,Drug Rebate Offset - National,2015
296,-842686786.00,Drug Rebate Offset - State Sidebar Agreement,2015
297,-10990020714.00,MCO - National Agreement,2015
298,-207156105.00,MCO - State Sidebar Agreement,2015


Calculate the total drug rebates by year.

In [15]:
drug_rebates = drug_rebates.groupby("year")["drug_rebates"].sum().to_frame().reset_index()
drug_rebates

,year,drug_rebates
0,2008,-8393320183.00
1,2009,-9721492081.00
2,2010,-11505122505.00
3,2011,-16117488352.00
4,2012,-17549478009.00
5,2013,-18274357490.00
6,2014,-19878420141.00
7,2015,-24030295066.00
8,2016,-31192868310.00


## Calculate drug spending less rebates

Join the drug spending and drug rebates dataframes.

In [16]:
drugs = drug_spending.merge(drug_rebates, on="year")
# Add the columns because the rebates are expressed as negatives
drugs["spending_less_rebates"] = drugs["drug_spending"] + drugs["drug_rebates"]
drugs

,year,drug_spending,drug_rebates,spending_less_rebates
0,2016,62016683226.54,-31192868310.00,30823814916.54
1,2015,55620114965.09,-24030295066.00,31589819899.09
2,2014,47078114991.53,-19878420141.00,27199694850.53
3,2013,37940450109.63,-18274357490.00,19666092619.63
4,2012,37976860399.61,-17549478009.00,20427382390.61
5,2011,37783429301.78,-16117488352.00,21665940949.78
6,2010,32989906941.56,-11505122505.00,21484784436.56
7,2009,26014836021.22,-9721492081.00,16293343940.22
8,2008,24642404374.76,-8393320183.00,16249084191.76


## Import data on Medicaid enrollment from [KFF](https://www.kff.org/)

In [17]:
enrollment = pd.read_excel("data/medicaid_spending/enrollment.xlsx", names=["year", "enrollment"])
enrollment

,year,enrollment
0,2008,59523604
1,2009,62692693
2,2010,66586779
3,2011,68039582
4,2012,71410849
5,2013,72332368
6,2014,80694100
7,2015,76041000
8,2016,76131500


Join the drugs data to the enrollment data.

In [18]:
spending_per_enrollee = enrollment.merge(drugs, on="year")
spending_per_enrollee.drop(["drug_spending", "drug_rebates"], axis=1, inplace=True)
spending_per_enrollee

,year,enrollment,spending_less_rebates
0,2008,59523604,16249084191.76
1,2009,62692693,16293343940.22
2,2010,66586779,21484784436.56
3,2011,68039582,21665940949.78
4,2012,71410849,20427382390.61
5,2013,72332368,19666092619.63
6,2014,80694100,27199694850.53
7,2015,76041000,31589819899.09
8,2016,76131500,30823814916.54


## Calculate drug spending per enrollee

In [19]:
spending_per_enrollee["spending_per_enrollee"] = spending_per_enrollee["spending_less_rebates"] / spending_per_enrollee["enrollment"]
spending_per_enrollee

,year,enrollment,spending_less_rebates,spending_per_enrollee
0,2008,59523604,16249084191.76,272.99
1,2009,62692693,16293343940.22,259.89
2,2010,66586779,21484784436.56,322.66
3,2011,68039582,21665940949.78,318.43
4,2012,71410849,20427382390.61,286.05
5,2013,72332368,19666092619.63,271.89
6,2014,80694100,27199694850.53,337.07
7,2015,76041000,31589819899.09,415.43
8,2016,76131500,30823814916.54,404.88


## Export the data for visualization and further analysis

In [20]:
writer = pd.ExcelWriter("data/medicaid_spending.xlsx")
budgets.to_excel(writer, "Budgets", startcol=0, index=False)
drugs.to_excel(writer, "Drugs", startcol=0, index=False)
spending_per_enrollee.to_excel(writer, "Drug Spending Per Enrollee", startcol=0, index=False)
writer.save()